In [ ]:
import os
import openai
import pinecone
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from json import JSONDecodeError
from PIL import Image
from io import BytesIO

#INSERT API KEYS HERE
pinecone.init(api_key="", environment="us-west4-gcp")
index = pinecone.Index(index_name='')
os.environ['OPENAI_API_KEY'] = ""
openai.api_key = ""

i=1


In [ ]:
#UPSERT CHUNKS INTO DB
import json

#edit this to add filenames and channel names of channels to be upserted
filenames = {
  "user-feedback.json": "#user-feedback", #<-- edit filenames here
}

for filename in filenames:

    with open(f"./to_pinecone/{filename}", 'r') as f:
        chunks = json.load(f)
    
    CHANNEL_NAME = filenames[filename]
    embed_id = 1

    for file in chunks:
        EMBED_ID= f"{CHANNEL_NAME}_{embed_id}"

        try:
            qa = file['qa']
        except KeyError:
            print(CHANNEL_NAME, "no qa")
            qa = file['messages']

        chunk = file['messages']
        link = file['link']
        time = file['time']

        timestamp = file['timestamp']

        description_response = openai.Embedding.create(
            input=qa,
            model="text-embedding-ada-002",
        )
        description_embeddings = description_response["data"][0]["embedding"]

        index.upsert([
        (EMBED_ID, description_embeddings, {"link": link, "chunk": chunk, "time": time, "timestamp": timestamp, "channel": CHANNEL_NAME })
        ], namespace='slack',)
        embed_id+=1
    print(f"{CHANNEL_NAME} ({embed_id} chunks)")


In [ ]:
#Test chunks 
QUERY = " Current status of Offer Letter e2e?"

response = openai.Embedding.create(
input=QUERY,
model="text-embedding-ada-002",
)
embeddings_vector = response["data"][0]["embedding"]

pinecone_result = index.query(vector=embeddings_vector,
                            include_metadata=True,
                            # filter=        {"channel": {
                            #         "$eq": "#bugs"
                            #       }},
                            top_k=15,
                            namespace='slack')
